# The book-reading service research

Our company decided to be on the wave and bought a large service for reading books by subscription. Our first job as an analyst is to analyze the database.
It contains information about books, publishers, authors, as well as user reviews of books. This data will help formulate a value proposition for a new product.

## Importing Pandas and SQL

In [1]:
import pandas as pd

In [2]:
from sqlalchemy import create_engine

db_config = {'user': 'praktikum_student', 
'pwd': 'Sdf4$2;d-d30pp', 
'host': 'rc1b-wcoijxj3yxfsf3fs.mdb.yandexcloud.net',
'port': 6432, 
'db': 'data-analyst-final-project-db'} 
connection_string = 'postgresql://{}:{}@{}:{}/{}'.format(db_config['user'],
 db_config['pwd'],
 db_config['host'],
 db_config['port'],
 db_config['db'])

engine = create_engine(connection_string, connect_args={'sslmode':'require'}) 

*We will check the work with simple queries, display the first rows of all tables*

In [3]:
query = '''
SELECT *
FROM Books
LIMIT 5;
''' 

In [4]:
pd.io.sql.read_sql(query, con = engine) 

,book_id,author_id,title,num_pages,publication_date,publisher_id
0,1,546,'Salem's Lot,594,2005-11-01,93
1,2,465,1 000 Places to See Before You Die,992,2003-05-22,336
2,3,407,13 Little Blue Envelopes (Little Blue Envelope...,322,2010-12-21,135
3,4,82,1491: New Revelations of the Americas Before C...,541,2006-10-10,309
4,5,125,1776,386,2006-07-04,268


*With a separate query, we display the total number of table rows*


In [5]:
query = '''SELECT COUNT(1) FROM Books;''' 

In [6]:
pd.io.sql.read_sql(query, con = engine) 

,count
0,1000


In [7]:
query = '''
SELECT *
FROM authors
LIMIT 5
''' 

In [8]:
pd.io.sql.read_sql(query, con = engine)

,author_id,author
0,1,A.S. Byatt
1,2,Aesop/Laura Harris/Laura Gibbs
2,3,Agatha Christie
3,4,Alan Brennert
4,5,Alan Moore/David Lloyd


In [9]:
query = '''SELECT COUNT(1) FROM authors;''' 

In [10]:
pd.io.sql.read_sql(query, con = engine) 

,count
0,636


In [11]:
query = '''
SELECT *
FROM publishers
LIMIT 5
''' 

In [12]:
pd.io.sql.read_sql(query, con = engine)

,publisher_id,publisher
0,1,Ace
1,2,Ace Book
2,3,Ace Books
3,4,Ace Hardcover
4,5,Addison Wesley Publishing Company


In [13]:
query = '''SELECT COUNT(1) FROM publishers;''' 

In [14]:
pd.io.sql.read_sql(query, con = engine) 

,count
0,340


In [15]:

query = '''
SELECT *
FROM reviews
LIMIT 5
''' 

In [16]:
pd.io.sql.read_sql(query, con = engine)

,review_id,book_id,username,text
0,1,1,brandtandrea,Mention society tell send professor analysis. ...
1,2,1,ryanfranco,Foot glass pretty audience hit themselves. Amo...
2,3,2,lorichen,Listen treat keep worry. Miss husband tax but ...
3,4,3,johnsonamanda,Finally month interesting blue could nature cu...
4,5,3,scotttamara,Nation purpose heavy give wait song will. List...


In [17]:
query = '''SELECT COUNT(1) FROM reviews;''' 

In [18]:
pd.io.sql.read_sql(query, con = engine) 

,count
0,2793


In [19]:

query = '''
SELECT *
FROM ratings
LIMIT 5
''' 

In [20]:
pd.io.sql.read_sql(query, con = engine)

,rating_id,book_id,username,rating
0,1,1,ryanfranco,4
1,2,1,grantpatricia,2
2,3,1,brandtandrea,5
3,4,2,lorichen,3
4,5,2,mariokeller,2


In [21]:
query = '''SELECT COUNT(1) FROM ratings;''' 

In [22]:
pd.io.sql.read_sql(query, con = engine) 

,count
0,6456


<p align="center"><img src='https://concrete-web-bad.notion.site/image/https%3A%2F%2Fs3-us-west-2.amazonaws.com%2Fsecure.notion-static.com%2F069818d1-0e5c-4d87-a461-0de584ab9c33%2FUntitled_(33).png?table=block&id=bd53c8db-b4fd-49eb-8cc9-572ebb3c9163&spaceId=9e4bd47b-c6e6-4ca3-bcee-279794b47315&width=1640&userId=&cache=v2'></p>


### Description of data

**Table `books`**

Contains information about books:

- `book_id` — book identifier;
- `author_id` — author identifier;
- `title` — title of the book;
- `num_pages` - number of pages;
- `publication_date` - publication date of the book;
- `publisher_id` - publisher identifier.

**The `authors` table**

Contains information about the authors:

- `author_id` — author identifier;
- `author` is the name of the author.

**Table `publishers`**

Contains information about publishers:

- `publisher_id` — publisher identifier;
- `publisher` — publisher name;

**Table `ratings`**

Contains data about user ratings of books:

- `rating_id` — rating identifier;
- `book_id` — book identifier;
- `username` - the name of the user who left the rating;
- `rating` - rating of the book.

**Table `reviews`**

Contains data about user reviews of books:

- `review_id` — review identifier;
- `book_id` — book identifier;
- `username` - the name of the user who wrote the review;
- `text` — review text.

*Let's look at other tables in the database and what types of data are in the tables of interest to us*

In [23]:

display(pd.io.sql.read_sql('''

SELECT *
FROM pg_catalog.pg_tables
WHERE schemaname != 'pg_catalog' AND
      schemaname != 'information_schema';

''', con = engine))


display(pd.io.sql.read_sql('''
SELECT 
    table_name, 
    column_name, 
    data_type, 
    is_nullable
FROM INFORMATION_SCHEMA.COLUMNS 
WHERE table_name IN ('books', 'authors', 'publishers', 'ratings', 'reviews');
''', con = engine))

,schemaname,tablename,tableowner,tablespace,hasindexes,hasrules,hastriggers,rowsecurity
0,public,orders,praktikum_admin,None,True,False,False,False
1,public,visits,praktikum_admin,None,True,False,False,False
2,public,advertisment_costs,praktikum_admin,None,True,False,False,False
3,public,authors,praktikum_admin,None,True,False,True,False
4,public,publishers,praktikum_admin,None,True,False,True,False
5,public,reviews,praktikum_admin,None,True,False,True,False
6,public,ratings,praktikum_admin,None,True,False,True,False
7,public,books,praktikum_admin,None,True,False,True,False


,table_name,column_name,data_type,is_nullable
0,authors,author_id,integer,NO
1,authors,author,text,YES
2,publishers,publisher_id,integer,NO
3,publishers,publisher,text,YES
4,reviews,review_id,integer,NO
5,reviews,book_id,integer,YES
6,reviews,username,text,YES
7,reviews,text,text,YES
8,ratings,rating_id,integer,NO
9,ratings,book_id,integer,YES


## Research

### Let's count how many books came out after January 1, 2000;


In [24]:
query = '''
SELECT COUNT (*)
FROM books
WHERE publication_date >=  '2000-01-01'
''' 

In [25]:
pd.io.sql.read_sql(query, con = engine)

,count
0,821


*Since January 1, 2000, 821 books have been published*

### For each book, we will calculate the number of reviews and the average rating;

In [26]:
query = '''
WITH rat AS 
(SELECT book_id,
ROUND (AVG (rating), 3) AS average_rating
FROM ratings
GROUP BY book_id),
rew AS 
(SELECT book_id,
COUNT (DISTINCT review_id) AS review_number
FROM reviews
GROUP BY book_id)

SELECT b.title,
       a.author,
        rat.average_rating,
        rew.review_number
       
FROM books as b 
LEFT JOIN rat ON b.book_id = rat.book_id
LEFT JOIN rew ON b.book_id = rew.book_id
LEFT JOIN authors a ON  a.author_id = b.author_id
ORDER BY average_rating DESC, review_number DESC
Limit 10
''' 

In [27]:
pd.io.sql.read_sql(query, con = engine)

,title,author,average_rating,review_number
0,The Cat in the Hat and Other Dr. Seuss Favorites,Dr. Seuss/Various,5.0,NaN
1,A Dirty Job (Grim Reaper #1),Christopher Moore,5.0,4.0
2,Moneyball: The Art of Winning an Unfair Game,Michael Lewis,5.0,3.0
3,School's Out—Forever (Maximum Ride #2),James Patterson,5.0,3.0
4,Captivating: Unveiling the Mystery of a Woman'...,John Eldredge/Stasi Eldredge,5.0,2.0
5,Alas Babylon,Pat Frank,5.0,2.0
6,Crucial Conversations: Tools for Talking When ...,Kerry Patterson/Joseph Grenny/Ron McMillan/Al ...,5.0,2.0
7,Act of Treason (Mitch Rapp #9),Vince Flynn,5.0,2.0
8,A Fistful of Charms (The Hollows #4),Kim Harrison,5.0,2.0
9,A Woman of Substance (Emma Harte Saga #1),Barbara Taylor Bradford,5.0,2.0


*Мы получили топ-10 Самых популярных книг в нашем сервисе*

### Let's determine the publisher has released the largest number of books

We will take only books thicker than 50 pages - exclude brochures from the analysis

In [28]:
query = '''
WITH big AS
(SELECT book_id,
publisher_id
FROM books
WHERE num_pages>50)

SELECT p.publisher,
COUNT (DISTINCT big.book_id) AS book_number
FROM publishers p
LEFT JOIN big ON p.publisher_id = big.publisher_id
GROUP BY p.publisher
ORDER BY book_number DESC
Limit 10
''' 

In [29]:
pd.io.sql.read_sql(query, con = engine)

,publisher,book_number
0,Penguin Books,42
1,Vintage,31
2,Grand Central Publishing,25
3,Penguin Classics,24
4,Ballantine Books,19
5,Bantam,19
6,Berkley,17
7,Berkley Books,14
8,St. Martin's Press,14
9,Delta,13


*Received the most prescriptive publishing houses*

### Let's determine the author with the highest average book rating

*We will take only books with 50 grades or more*

In [30]:
query = '''
SELECT a.author,
AVG (r.rating) AS average_rating
FROM authors a
LEFT JOIN books b ON a.author_id = b.author_id
LEFT JOIN ratings r ON b.book_id = r.book_id
WHERE b.book_id in  (SELECT b.book_id
                   FROM books b 
                   LEFT JOIN ratings r ON b.book_id = r.book_id 
                   GROUP BY b.book_id 
                   HAVING COUNT (DISTINCT r.rating_id) >=50)

GROUP BY a.author
ORDER BY average_rating DESC
LIMIT 1
''' 

In [31]:
pd.io.sql.read_sql(query, con = engine)

,author,average_rating
0,J.K. Rowling/Mary GrandPré,4.287097


*Thus, the highest-rated author is Harry Potter author J.K. Rowling with illustrations by Mary GrandPré (average rating of their books 4.29)*

### Let's calculate the average number of reviews from users

We will consider only users who have put in more than 50 grades.

In [32]:
query = '''
SELECT AVG (group_rev.review_number)
FROM (SELECT username,
      COUNT (DISTINCT review_id) AS review_number
      FROM reviews
      WHERE username in (SELECT username
                         FROM ratings
                         GROUP BY username
                         HAVING COUNT (DISTINCT rating_id) >=50)
      GROUP BY username) AS group_rev

''' 

In [33]:
pd.io.sql.read_sql(query, con = engine)

,avg
0,24.222222


*The active service users average write 24 reviews*

## **CONCLUSIONS**


1. We examined the database of the book reading service.
2. Got the most popular books to promote on promo.
3. We found the most productive publishing houses for the convenience of buying rights to books.
4. The most beloved Author was determined - JK Rowling.
5. We found out how many reviews are written by active users on average - 24
6. Find out how many books in our library have been published since January 1, 2001